In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import psutil 
import gc

In [2]:
cd C:/Users/minih/python_prac1/m5/git-repository/kaggle-m5-accuracy-library/library/

C:\Users\minih\python_prac1\m5\git-repository\kaggle-m5-accuracy-library\library


In [3]:
# reduce用
import patair

In [4]:
cd C:/Users/minih/python_prac1/library/m5/

C:\Users\minih\python_prac1\library\m5


In [5]:
def encode_categorical(df, cols):
    
    for col in cols:
        # Leave NaN as it is.
        le = LabelEncoder()
        #not_null = df[col][df[col].notnull()]
        df[col] = df[col].fillna('nan')
        df[col] = pd.Series(le.fit_transform(df[col]), index=df.index)

    return df


In [22]:
datas = pd.read_csv("add_Clean_Analysis_WI_3.csv").pipe(patair.reduce_mem_usage)
datas = encode_categorical(
   datas, ["item_id"]
).pipe(patair.reduce_mem_usage)
gc.collect()

Mem. usage decreased to 119.65 Mb (50.0% reduction)
Mem. usage decreased to 99.71 Mb (6.2% reduction)


98

In [23]:
datas.sort_values(["item_id", "variable", "week", "wday"], inplace=True)

In [24]:
datas["variable"] = datas["variable"].apply(lambda x: x[-2:])
datas["variable"] = datas["variable"].astype(np.int8)

In [25]:
datas.drop("Unnamed: 0", axis=1, inplace=True)
datas = datas[datas["variable"] >= 13]

In [26]:
#順序の整った一意なキーを取得
product = pd.read_pickle("product_WI_3.pickle")

In [20]:
datas

,item_id,week,wday,variable,value,prev_1year,year,date
1612,0,1,1,13,0.0,NaN,2013,2013-01-26
4661,0,1,2,13,1.0,NaN,2013,2013-01-27
7710,0,1,3,13,4.0,NaN,2013,2013-01-28
10759,0,1,4,13,3.0,NaN,2013,2013-01-29
13808,0,1,5,13,1.0,NaN,2013,2013-01-30
...,...,...,...,...,...,...,...,...
3471373,3048,16,5,16,NaN,0.0,2016,2016-05-18
3474422,3048,16,6,16,NaN,0.0,2016,2016-05-19
3477471,3048,16,7,16,NaN,0.0,2016,2016-05-20
3480520,3048,17,1,16,NaN,0.0,2016,2016-05-21


In [27]:
#一意なキーをnumpy配列で取得
columns = np.array(product["item_id"])

In [22]:
columns

array([1437, 1438, 1439, ..., 1434, 1435, 1436], dtype=int16)

In [28]:
#各アイテムIDについて、0が15日以上続く区間で昨年の売り上げ(prev365)を射影
for l, item in enumerate(columns):
    buff = datas[datas["item_id"] == item]
    buff.reset_index(inplace=True)
    array = np.array(buff["value"])
    j = 0
    range_list = []
    for i in range(len(array)):
        if array[i] == 0:
            j += 1
        else:
            if j >= 15:
                range_list.extend(np.arange(i-j,i,1))
            j = 0
    if j >= 15:
        range_list.extend(np.arange(i-j+1,i+1,1))
    change_array = np.array(buff.loc[range_list]["index"])
    datas.loc[change_array,"value"] = datas.loc[change_array,"prev_1year"]
    if l % 100 == 0:
        print(f"{l}/3049")   

0/3049
100/3049
200/3049
300/3049
400/3049
500/3049
600/3049
700/3049
800/3049
900/3049
1000/3049
1100/3049
1200/3049
1300/3049
1400/3049
1500/3049
1600/3049
1700/3049
1800/3049
1900/3049
2000/3049
2100/3049
2200/3049
2300/3049
2400/3049
2500/3049
2600/3049
2700/3049
2800/3049
2900/3049
3000/3049


In [29]:
datas.to_pickle("WI_3_interpolate.pickle")